In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster
from geojson import MultiLineString
from folium.plugins import HeatMap
from glob import glob
from folium import plugins


In [2]:
# readind data from csv file into dataframe
traffic_vol_csv_all = pd.read_csv("Traffic_Volumes_for_2018.csv")

# filtering data for year 2018
traffic_vol_csv = traffic_vol_csv_all[traffic_vol_csv_all['YEAR'] == 2018]

# remove own index with default index 
traffic_vol_csv.reset_index(inplace = True, drop = True)

traffic_vol_csv.head()


,YEAR,SECNAME,Shape_Leng,VOLUME,multilinestring
0,2018,5AVS12,172.570274,22000,MULTILINESTRING ((-114.06036700906716 51.04831...
1,2018,94AVSEWBON,189.451491,5000,MULTILINESTRING ((-114.06876825342002 50.96863...
2,2018,12STNE29,261.453939,5000,MULTILINESTRING ((-114.03372206187294 51.05323...
3,2018,100AVNE1,804.353006,2000,MULTILINESTRING ((-114.01318801277267 51.14336...
4,2018,106AVSE1,507.320869,14000,MULTILINESTRING ((-113.98128756128924 50.95784...


In [3]:
# refining data in multilinestring to fetch individual coordinates
traffic_vol_csv['multilinestring'] = traffic_vol_csv['multilinestring'].apply(lambda x: x[16:].replace("(","").replace(")",""))

# creating lists for latitude, longitude, and volume for 'traffic_vol_csv'
vol_lat_list =[]
vol_long_list = []
vol_list = []

# algo for getting lats, longs, and volume and appending the values in the list below
for i in range (traffic_vol_csv.shape[0]):
    coordinates = traffic_vol_csv.loc[i,'multilinestring'].split(",")
    for j in coordinates:
        cord_space = j.split()
        vol_long_list.append(float(cord_space[0]))
        vol_lat_list.append(float(cord_space[1]))
        vol_list.append(float(traffic_vol_csv.loc[i,'VOLUME']))
    
print(len(vol_list))
print(len(vol_long_list))
print(len(vol_lat_list))


C:\Users\nehas\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


41607
41607
41607


In [9]:
# creating new dataframe with final data consisting of individual coordinates and loading the data from the lists
df_volume = pd.DataFrame({'VOLUME': vol_list, 'latitude':vol_lat_list, 'longitude': vol_long_list })

# computing total volume for each coordinate
# we group the data based on lats and longitude and take the sum of volume for each.
df_volume[['latitude', 'longitude', 'VOLUME']].groupby(['latitude', 'longitude' ]).sum()

,,VOLUME
latitude,longitude,
50.848539,-114.023095,21000.0
50.848621,-114.023161,21000.0
50.848703,-114.023226,21000.0
50.848795,-114.023300,21000.0
50.848877,-114.023366,21000.0
...,...,...
51.211614,-114.094579,0.0
51.211884,-114.094579,0.0
51.212155,-114.094580,0.0


In [10]:
# verifying datatypes of columns
# df_volume.dtypes

In [11]:
# creating and loading the map of Calgary
Calgary_CD = folium.Map(location = [51.049999, -114.066666], zoom_start=10)

# Adding heatmap to the Calgary map based on coordinates
# radius manages teh density of the plot, zoom value defines the initial zoom
HeatMap(df_volume[['latitude', 'longitude', 'VOLUME']].groupby(['latitude', 'longitude' ]).sum().reset_index(),radius=8,  max_zoom=4).add_to(Calgary_CD)

In [12]:
# To show the map in jupyter
Calgary_CD